Spark Transforming(데이터 변환)

    Data -----> ML(machine learning)
    연설문      패턴
    대여건수    알고리듬
    
        변환 과정에서 필요한 요소 : label, features (question마다 다름)
        label : 구분(분류)/판단, features : 구분/판단근거의 특징
          ex) 사망자의 데이터를 변환할 때,
              label : 사망 or 생존, features : 사망의 근거(원인)
              
        label의 다른 이름은 Class.
        
    
    Spark ETL : Extract, Transform, Loading
        => Spark Transform으로 부르겠다.


Vectors

numpy와 Vector는 같은 용도로 쓰이나, numpy는 단일로, Vector는 분산처리가 가능한 처리방식이다.

    Matrix : 2차원 Vector
    
    Vector를 쓰기 위해서는 ml or mllib를 사용.
        mllib : Rdd API
        ml : DataFrame API
        
    Vector는 dense와 sparse로 분류.
        dense : 0을 포함하는 Vector
        sparse : 0을 제거한 Vector

In [1]:
import os
import sys

os.environ["PYLIB"]=os.path.join(os.environ["SPARK_HOME"],'python','lib')
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'py4j-0.10.1-src.zip'))
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'pyspark.zip'))

import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config("spark.sql.warehouse.dir", "C:/Users/G312")\
    .getOrCreate()

In [3]:
import numpy as np

In [6]:
dv=np.array([1,0,2,1,3])

In [7]:
type(dv) #numpy를 통해 만든 어레이 리스트

numpy.ndarray

In [8]:
from pyspark.mllib.linalg import Vectors

In [9]:
dv1mllib=Vectors.dense([1.0, 2.1, 3])
print dv1mllib, type(dv1mllib)

[1.0,2.1,3.0] <class 'pyspark.mllib.linalg.DenseVector'>


linalg = Linear Algebra(선형대수학)

위의 작업을 통해 dense인 vector를 생성하였다.

dense vectors는 numpy array와 같은 특징을 가진다.

In [10]:
from pyspark.ml.linalg import Vectors

dv1ml=Vectors.dense([1.0, 2.1, 3])
print dv1ml

[1.0,2.1,3.0]


In [11]:
for e in dv1ml:
    print e, #반복문 사용 가능

1.0 2.1 3.0


In [12]:
print dv1ml.dot(dv1ml) # 벡터 연산 가능

14.41


In [13]:
np.dot(dv,dv)

15

In [14]:
print dv1ml*dv1ml

[1.0,4.41,9.0]


In [15]:
sv1 = Vectors.sparse(3, [1, 2], [1.0, 3.0])
print sv1.toArray()

[0. 1. 3.]


Sparse Vectors는 0이 포함된 경우 이를 생략하여 자원의 공간을 줄인다.

toArray를 통해 dense로 벡터를 변환할 수 있다.

In [16]:
import numpy as np
import scipy.sparse as sps

row = np.array([0, 0, 1, 2, 2, 2])
col = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])
mtx = sps.csc_matrix((data, (row, col)), shape=(3, 3))
print mtx.todense()   


[[1 0 2]
 [0 0 3]
 [4 5 6]]


row, col, data를 통해 Compressed Sparse Column data를 생성한 모습이다.
row, col은 행과 열의 좌표, data는 해당하는 좌표의 value를 의미한다.

In [17]:
p = [[1.0,2.0,3.0],[1.1,2.1,3.1],[1.2,2.2,3.3]]
my=spark.sparkContext.parallelize(p)

my.collect() 

[[1.0, 2.0, 3.0], [1.1, 2.1, 3.1], [1.2, 2.2, 3.3]]

In [19]:
from pyspark.mllib.linalg.distributed import RowMatrix
rm=RowMatrix(my)
#RowMatrix를 이용하면 여러 행으로 구성된 벡터를 생성한다.
print type(rm)

rm.rows.collect()

<class 'pyspark.mllib.linalg.distributed.RowMatrix'>


[DenseVector([1.0, 2.0, 3.0]),
 DenseVector([1.1, 2.1, 3.1]),
 DenseVector([1.2, 2.2, 3.3])]

Labeled Point는 분류와 회귀분석에 사용되는 데이터 타입으로, label과 features로 구성

    Label : 지도학습에서 '구분 값'으로 활용, Double Type
    Features : sparse, dense 모두 사용 가능

In [21]:
from pyspark.mllib.regression import LabeledPoint

print LabeledPoint(1.0, [1.0, 2.0, 3.0])

#label = 1.0(double), features = [1.0, 2.0, 3.0](dense)

(1.0,[1.0,2.0,3.0])


In [22]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors

print LabeledPoint(1992, Vectors.sparse(10, {0: 3.0, 1:5.5, 2: 10.0}))


(1992.0,(10,[0,1,2],[3.0,5.5,10.0]))


처음에 패키지 부를 때 ml과 mllib를 혼동해서는 안된다.(위 참조)